# <center> <img src="../../img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> Computer Systems Engineering  </center>
---
### <center> Big Data Processing </center>
---
#### <center> **Autumn 2025** </center>

#### <center> **Final Project: Machine Learning** </center>
---

**Date**: November, 2025

**Student Name**: Juan Bernardo Orozco Quirarte

**Professor**: Pablo Camarillo Ramirez

# Creamos Sesion de Spark

In [1]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
from pyspark.sql.functions import count, col
from pyspark.ml.feature import StringIndexer, IndexToString
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator

spark = SparkSession.builder \
    .appName("ML: ALS") \
    .master("spark://spark-master:7077") \
    .config("spark.ui.port", "4040") \
    .getOrCreate()

sc = spark.sparkContext
sc.setLogLevel("INFO")

# Optimization (reduce the number of shuffle partitions)
spark.conf.set("spark.sql.shuffle.partitions", "5")

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/11/24 02:54:47 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# 1. Machine Learning Algorithm to use (10 pts)

**Problema Seleccionado:** Sistema de Recomendación (Recommendation System).

**Justificación:**
Elegí resolver un problema de recomendación porque es una de las aplicaciones más prácticas y comunes del Big Data hoy en día (como en Netflix o Amazon). El objetivo es predecir qué libros le gustarían a un usuario basándose en sus calificaciones pasadas, ayudando a filtrar información relevante en un catálogo inmenso.

**Algoritmo Seleccionado:**
Utilizaré **Alternating Least Squares (ALS)**. Este algoritmo de Filtrado Colaborativo es ideal para este problema porque factoriza la matriz de usuarios y productos para encontrar patrones latentes, y está optimizado en Spark para trabajar con grandes volúmenes de datos dispersos.

---

# 2. Dataset Description (20 pts)

**Fuente del Dataset:**
El dataset utilizado es el **"Book-Crossing Dataset"**, obtenido de [Kaggle](https://www.kaggle.com/ruchi798/bookcrossing-dataset).

**Contenido:**
El dataset original consta de tres archivos CSV: Usuarios, Libros (con metadatos como Título y Autor) y Calificaciones (Ratings).

**Estadísticas del Dataset (Analizadas con PySpark):**
Para este proyecto, nos enfocamos en el archivo de *Ratings*, filtrando calificaciones explícitas (escala 1-10). Despues de filtrar por usuarios con más de 3 ratings y libros con mas de 5 nos queda lo siguiente.

* **Tamaño del dataset (filas):** 1149780 registros crudos y 166646 registros despues de mi limpieza.
* **Usuarios únicos:** 19966 usuarios post limpieza.
* **Libros (Items) únicos:** 14520 libros distintos post limpiea.

**A Aclarar**: 
Hay más de un millon de registros y más de 100k usuarios y libros, pero hay libros sin ratings y usarios que no han calificado ningun libro, al igual que hay registros con rating 0, los rating con 0 no son ratings explicitamente, solo es un indicativo que el usario compro el libro pero no lo ha calificado ahun, por lo que no nos sirven esos registros. Por eso se reduce el tamaño del dataset, la limpieza nos ayuda a hacerlo más preciso para usuarios activos y libros calificados y asi evitar sesgos. 

**Nota sobre los datos:**
Al ser un sistema de recomendación, la métrica clave es la interacción Usuario-Libro. Los datos representan feedback explícito (estrellas del 1 al 10), lo cual permite al modelo predecir una calificación exacta.

In [2]:
# Carga del dataset y definicion del esquema
from bernardoorozco.spark_utils import SparkUtils

schema_cols = [("User-ID", "int"), ("ISBN", "string"), ("Book-Rating", "int")]
schema= SparkUtils.generate_schema(schema_cols)


df_ratings = spark.read \
                    .option("header", "true") \
                    .option("delimiter", ";") \
                    .schema(schema) \
                    .csv("/opt/spark/work-dir/lib/bernardoorozco/ml/BX-Book-Ratings.csv")  

print(f"Total de registros crudos {df_ratings.count()}")

Total de registros crudos 1149780


## Limpieza y filtrado

In [3]:
df_ratings = df_ratings \
    .withColumnRenamed("User-ID", "user_idf") \
    .withColumnRenamed("ISBN", "isbn_str") \
    .withColumnRenamed("Book-Rating", "rating")
df_clean = df_ratings.filter(col("rating") > 0) # Rating = 0 representa mas del 50% del dataset porque 0 son compras del libro mas no ratings

book_counts = df_clean.groupBy("isbn_str").agg(count("rating").alias("book_count"))
popular_books = book_counts.filter(col("book_count") >= 5) # Minimo 5 ratings por libro

user_counts = df_clean.groupBy("user_idf").agg(count("rating").alias("user_count"))
active_users = user_counts.filter(col("user_count") >= 3) # Minimo 3 ratings por user

# Solo nos quedamos con la interseccion
df_optimized = df_clean \
    .join(popular_books, "isbn_str", "inner") \
    .join(active_users, "user_idf", "inner") \
    .select(df_clean["user_idf"], df_clean["isbn_str"], df_clean["rating"])

print(f"Total calificaciones (1-10): {df_optimized.count()}")
print(f"Usuarios únicos: {df_optimized.select('user_idf').distinct().count()}")
print(f"Libros únicos: {df_optimized.select('isbn_str').distinct().count()}")

Total calificaciones (1-10): 166646


Usuarios únicos: 19966


Libros únicos: 14520


## Preprocesamiento

In [4]:
# ISBN a integer e indices numericos a usuarios
from pyspark.ml import Pipeline


indexer_user = StringIndexer(inputCol="user_idf", outputCol="user_id", handleInvalid="skip")
indexer_book = StringIndexer(inputCol="isbn_str", outputCol="book_id", handleInvalid="skip")

pipeline = Pipeline(stages=[indexer_user, indexer_book])

print("Indexando usuarios y libros...")
model_pipeline = pipeline.fit(df_optimized)
df_final = model_pipeline.transform(df_optimized)

Indexando usuarios y libros...


# 3. ML Training process (30 points)

Justificación de Hiperparámetros:

rank=20 y maxIter=20: Elegí estos valores para darle suficiente complejidad al modelo y tiempo para aprender, sin que tarde demasiado en procesar.

regParam=0.25: Usé una regularización un poco alta para filtrar el ruido, ya que muchos usuarios califican pocos libros y eso puede confundir al modelo.

implicitPrefs=False: Fundamental, porque estamos usando calificaciones reales (1 a 10) y no datos implícitos como clics.

coldStartStrategy="drop": Para que la evaluación no falle si aparecen usuarios nuevos en el test.

In [5]:
# Split
training, test= df_final.randomSplit([0.8, 0.2], seed=42)

# ALS
als = ALS(
    userCol="user_id", 
    itemCol="book_id", 
    ratingCol="rating", 
    implicitPrefs=False,
    nonnegative=True,
    coldStartStrategy="drop",
    rank=20, 
    maxIter=20, 
    regParam=0.25
)

# Entrenamiento
print("Entrenando modelo de libros...")
model = als.fit(training)

Entrenando modelo de libros...


25/11/24 02:55:59 WARN DAGScheduler: Broadcasting large task binary with size 1200.2 KiB
25/11/24 02:56:00 WARN DAGScheduler: Broadcasting large task binary with size 1201.7 KiB
25/11/24 02:56:01 WARN DAGScheduler: Broadcasting large task binary with size 1203.2 KiB
25/11/24 02:56:01 WARN DAGScheduler: Broadcasting large task binary with size 1204.6 KiB
25/11/24 02:56:02 WARN DAGScheduler: Broadcasting large task binary with size 1203.5 KiB
25/11/24 02:56:03 WARN DAGScheduler: Broadcasting large task binary with size 1204.9 KiB
25/11/24 02:56:04 WARN DAGScheduler: Broadcasting large task binary with size 1205.7 KiB
25/11/24 02:56:05 WARN DAGScheduler: Broadcasting large task binary with size 1209.1 KiB
25/11/24 02:56:06 WARN DAGScheduler: Broadcasting large task binary with size 1210.6 KiB
25/11/24 02:56:07 WARN DAGScheduler: Broadcasting large task binary with size 1212.1 KiB
25/11/24 02:56:08 WARN DAGScheduler: Broadcasting large task binary with size 1213.7 KiB
25/11/24 02:56:09 WAR

## Persistencia modelo

In [6]:
# Guardar el modelo
model_path = "/opt/spark/work-dir/data/mlmodels/als/als_books"
model.write().overwrite().save(model_path)
print(f"Modelo guardado en: {model_path}")

25/11/24 02:56:42 WARN DAGScheduler: Broadcasting large task binary with size 1500.1 KiB
25/11/24 02:56:51 WARN DAGScheduler: Broadcasting large task binary with size 1498.6 KiB


Modelo guardado en: /opt/spark/work-dir/data/mlmodels/als/als_books


# 4. ML Evaluation (20 points)

Métrica utilizada: Elegí RMSE (Root Mean Square Error).

¿Por qué? Porque estoy prediciendo un número exacto (estrellas del 1 al 10). El RMSE me dice, en promedio, por cuántos puntos se equivocó mi predicción respecto a la realidad.

Resultado: Obtuve un error de 1.88.

Interpretación: Significa que el modelo suele desviarse menos de 2 estrellas de la calificación real, lo cual es un resultado decente considerando lo subjetivos que son los gustos en libros.

In [7]:
from pyspark.ml.recommendation import ALSModel


# Cargar el modelo desde el disco
print(f"Cargando modelo desde: {model_path}...")
loaded_model = ALSModel.load(model_path)


predictions = loaded_model.transform(test)

evaluator = RegressionEvaluator(
    metricName="rmse", 
    labelCol="rating", 
    predictionCol="prediction"
)

rmse = evaluator.evaluate(predictions)
print(f"RMSE (Error promedio en estrellas): {rmse:.2f}")

# Muestra visual
print("Predicción vs Realidad:")
predictions.select("user_id", "isbn_str", "rating", "prediction").show(30)

Cargando modelo desde: /opt/spark/work-dir/data/mlmodels/als/als_books...


25/11/24 02:57:13 WARN DAGScheduler: Broadcasting large task binary with size 1228.0 KiB


RMSE (Error promedio en estrellas): 1.88
Predicción vs Realidad:


25/11/24 02:57:25 WARN DAGScheduler: Broadcasting large task binary with size 1220.8 KiB


+-------+----------+------+----------+
|user_id|  isbn_str|rating|prediction|
+-------+----------+------+----------+
|   8576|0553264990|     5| 3.3146055|
|  16447|0676973655|     3|  4.252397|
|  13305|8445071408|     7| 7.5595984|
|   7978|0316748641|     7|  5.307867|
|   5607|0451208080|     8|  8.249564|
|  10887|0061099325|     4| 5.9868307|
|   5255|3257233051|     9|  7.727029|
|   2151|0060977493|     7| 7.0600786|
|   2151|0446606383|     6| 6.6108456|
|   2151|0449006522|     6| 7.9352884|
|   2151|0553580388|     8| 7.3770123|
|   2151|155874262X|     5| 7.9744673|
|   4569|0060987103|     8| 7.5712585|
|   4569|0804111359|     8|  7.497422|
|    740|0142001740|     9| 7.6105204|
|    740|0380789035|    10|  7.520733|
|    740|0439064872|     9|  7.929495|
|   9490|0380815923|    10|  5.783345|
|   4116|1573225126|     5|  6.094401|
|   9525|0425156842|     7|  5.624893|
|   9525|0452277337|     7|  5.215217|
|   9525|0671021001|     6|  6.048205|
|   6346|039914739X|     

In [8]:
from pyspark.sql.functions import col

target_user = 276747 

print(f"Generando recomendaciones para el Usuario ID: {target_user}")

# Crear DataFrame solo con el usuario
request_df = spark.createDataFrame([(target_user,)], ["user_idf"])

try:
    # Extraemos el transformador de usuarios (Stage 0) para convertir el ID.
    user_indexer_model = model_pipeline.stages[0]
    request_indexed = user_indexer_model.transform(request_df)
    
    if request_indexed.count() == 0:
        print("Usuario desconocido para el modelo (No existe en los datos de entrenamiento).")
    else:
        # Generar 5 recomendaciones usando el indice interno
        recommendations = loaded_model.recommendForUserSubset(request_indexed, 5)
        
        # Para decodificar los libros necesitamos el Stage 1
        book_indexer_model = model_pipeline.stages[1] 
        book_labels = book_indexer_model.labels
        
        if recommendations.count() > 0:
            recs_row = recommendations.collect()[0]
            book_list = recs_row['recommendations'] 
            
            print("-" * 60)
            print(f"TOP 5 libros recomendados para: {target_user}")
            print("-" * 60)
            
            for item in book_list:
                book_idx = int(item['book_id'])
                rating = item['rating']
                
                final_rating = min(max(rating, 1.0), 10.0)  # Para que no se pase de 10 porque a ALS le vale pasarse
                
                # Recuperar ISBN real usando los labels del stage de libros
                real_isbn = book_labels[book_idx]
                
                print(f"ISBN: {real_isbn} | Predicción: {final_rating:.2f} / 10")
        else:
            print("El modelo no pudo generar recomendaciones para este usuario.")

except Exception as e:
    print(f"Error en demo: {e}")

Generando recomendaciones para el Usuario ID: 276747


------------------------------------------------------------
TOP 5 libros recomendados para: 276747
------------------------------------------------------------
ISBN: 0440471478 | Predicción: 10.00 / 10
ISBN: 0312099045 | Predicción: 10.00 / 10
ISBN: 0836213319 | Predicción: 10.00 / 10
ISBN: 0452261368 | Predicción: 10.00 / 10
ISBN: 0743467558 | Predicción: 10.00 / 10


In [9]:
sc.stop()